# Baseline with mse loss with wide cnn
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=6
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=6


Using TensorFlow backend.


In [2]:
window_size = 2001
process_all = False
sample_num = 1000

In [3]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [4]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# get intervals for day3 data
day3_intervals = list(BedTool(data.intervals['day3']))
print '# of Intervals Extracted for day3: {}'.format(len(day3_intervals))

# of Intervals Extracted for day0: 267226
# of Intervals Extracted for day3: 233846


In [5]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

# create an ArrayExtractor for ATAC-seq for day3 with 140 base pairs
bw_140bp_day3 = ArrayExtractor(data.input_atac['day3']['140'])
print 'Finished extracting bigwig for day3, 140bp'

Finished extracting bigwig for day0, 140bp
Finished extracting bigwig for day3, 140bp


In [6]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

# create a BigWigExtractor for histone makr 'H3K27ac' for day3
bw_histone_mark_day3 = BigwigExtractor(data.output_histone['day3']['H3K27ac'])
print 'Finished extracting bigwig for day3, 140bp'

Finished extracting bigwig for day0, 140bp
Finished extracting bigwig for day3, 140bp


In [7]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

# normalize day3 intervals
normalized_day3_intervals = [normalize_interval(interval, window_size) for interval in day3_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day3 intervals!'

Finished normalizing day0 intervals!
Finished normalizing day3 intervals!


In [8]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
assert (len(day3_intervals)==len(normalized_day3_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [9]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
atac_seq_day3 = bw_140bp_day3(normalized_day3_intervals)
print atac_seq_day0.shape, atac_seq_day3.shape

(267226, 2001, 5) (233846, 2001, 5)


In [10]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

print "Before pruning day3: {}".format(len(normalized_day3_intervals))
prune_invalid_intervals(normalized_day3_intervals, bw_140bp_day3)
print "After pruning day3: {}".format(len(normalized_day3_intervals))

Before pruning day0: 267226
After pruning day0: 267226
Before pruning day3: 233846
After pruning day3: 233846


In [11]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 2001, 5)


In [12]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 2001)


In [13]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

histone_mark_day3 = np.nan_to_num(bw_histone_mark_day3(normalized_day3_intervals))
print histone_mark_day3.shape

(267226, 2001)
(233846, 2001)


In [14]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
histone_mark_day3 = np.expand_dims(histone_mark_day3, axis=2)
print histone_mark_day0.shape, histone_mark_day3.shape

(267226, 2001, 1) (233846, 2001, 1)


In [15]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.01014  0.01014  0.01014  0.02435  0.02435]


In [16]:
from keras.layers import Input, Dense, Conv1D, Dropout
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [17]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 6
hidden_kernel_size_1 = 1000
# parameters for second conv layer
hidden_filters_2 = 6
hidden_kernel_size_2 = 600
# parameters for thirs conv layer
hidden_filters_3 = 6
hidden_kernel_size_3 = 400
# parameters for the output layer
output_filters = 1
output_kernel_size = 5
# parameters for training
batch_size = 64
num_epochs = 100
evaluation_freq = 10

In [18]:
'''
Baseline CNN (based on KERAS functional API)
'''
# 
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1)(inputs)
x = Dropout(dropout_rate)(x)


x = Conv1D(
    filters=hidden_filters_2,
    kernel_size=hidden_kernel_size_2,
    padding='same',
    activation='relu',
    strides=1)(x)
x = Dropout(dropout_rate)(x)

x = Conv1D(
    filters=hidden_filters_3,
    kernel_size=hidden_kernel_size_3,
    padding='same',
    activation='relu',
    strides=1)(x)
x = Dropout(dropout_rate)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [19]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2001, 5)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2001, 6)           30006     
_________________________________________________________________
dropout_1 (Dropout)          (None, 2001, 6)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2001, 6)           21606     
_________________________________________________________________
dropout_2 (Dropout)          (None, 2001, 6)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2001, 6)           14406     
_________________________________________________________________
dropout_3 (Dropout)          (None, 2001, 6)           0         
__________

In [20]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [21]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [25]:
model_dir = os.path.join("models", "mse_wide")
log_dir = os.path.join("logs", "mse_wide")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "mse_wide")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [26]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [27]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [28]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=7,
                              min_lr=1e-5)

In [29]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [30]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(7500, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:7500], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(2500, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[7500:10000], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(1000, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[10000:11000], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [31]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [32]:
print "Fitting the model..."
X_train, y_train = atac_seq_day0[:7500], histone_mark_day0[:7500]
X_val, y_val = atac_seq_day0[7500:10000], histone_mark_day0[7500:10000]
X_test, y_test = atac_seq_day0[10000:11000], histone_mark_day0[10000:11000]

model.fit(atac_seq_day0[:10000],
          histone_mark_day0[:10000],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.25,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 7500 samples, validate on 2500 samples
Epoch 1/100
7488/7500 [============================>.] - ETA: 0s - loss: 508.2389 - pearson: 0.0429 - mean_squared_error: 508.2389 - mean_absolute_error: 7.3228     - ETA: 11s - loss: 477.2352 - pearson: 3.8953e-04 - mean_squared_error: 477.2352 - mean_absolute_error: 7.1 - ETA: 10s - loss: 468.4601 - pearson: 3.6235e-04 - mean_squared_error: 468.4600 - mean_absolute_err - ETA: 9s - loss: 471.5143 - pearson: 0.0203 - mean_squared_error: 471.5142 - mean_absolute - ETA: 8s - loss: 459.1504 - pearson: 0.0178 - mean_squared_error: 459.1503 - mean_absolute_e - ETA: 7s - loss: 450.0127 - pearson: 0.0161 - mean_squared_error: 450.0126 - mean_absolute_error: 6. - ETA: 6s - loss: 448.0461 - pearson: 0.0156 - mean_squared_error: 448.0460 - mean_absolute_error: 6. - ETA: 6s - loss: 459.7206 - pearson: 0.0152 - mean_squared_error: 459.7205 - mean_absolute_err - ETA: 5s - loss: 465.4147 - pearson: 0.0280 - mean_squared_error: 465.

7488/7500 [============================>.] - ETA: 0s - loss: 499.3195 - pearson: 0.0427 - mean_squared_error: 499.3195 - mean_absolute_error: 7.2530 ETA: 8s - loss: 524.5015 - pearson: 0.0323 - mean_squared_error: 524.5015 - mean_absolute_error: 7.62 - ETA: 8s - loss: 543.5943 - pe - ETA: 4s - loss: 528.5886 - pearson: 0.0429 - mean_squared_error: 528.5886 - mean_absolute_error:  - ETA: 4s - loss: 536.0322 - pearson: 0.0411 - mean_squared_error: 536.0322 - mean_absolute_error: 7. - ETA: 4s - loss: 532
Epoch 00006: val_loss improved from 209.25684 to 208.55596, saving model to models/mse_wide/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/ms

/users/jesikmin/anaconda2/envs/genomelake_env/lib/python2.7/site-packages/numpy/lib/function_base.py:3167: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/users/jesikmin/anaconda2/envs/genomelake_env/lib/python2.7/site-packages/numpy/lib/function_base.py:3168: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/users/jesikmin/anaconda2/envs/genomelake_env/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/users/jesikmin/anaconda2/envs/genomelake_env/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/users/jesikmin/anaconda2/envs/genomelake_env/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Test spearman: nan

Epoch 00010: val_loss improved from 206.62178 to 206.03302, saving model to models/mse_wide/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============================] - 176s 23ms/step - loss: 493.1411 - pearson: 0.0512 - mean_squared_error: 493.1411 - mean_absolute_error: 7.3178 - val_loss: 206.0330 - val_pearson: 0.0016 - val_mean_squared_error: 206.0330 - val_mean_absolute_error: 3.9390
Epoch 11/100
7488/7500 [============================>.] - ETA: 0s - loss: 486.8874 - pearson: 0.0598 - mean_squared_error: 486.8875 - mean_absolute_error

Epoch 16/100
7488/7500 [============================>.] - ETA: 0s - loss: 484.9137 - pearson: 0.0085 - mean_squared_error: 484.9137 - mean_absolute_error: 7.49742284 - ETA: 11s - loss: 523.2185 - pearson: 9.3613e-16 - mean_squared_error: 523.2185 - mean_abso - ETA: 9s - loss: 479.1822 - pearson: 1.3114e-15 - mean_squared - ETA: 7s - loss: 431.1761 - pearson: 1.4455e-15 - mean_squared_error: 431.1761 - mean_absolute_error: 7. - ETA: 7s - loss: 425.7986 - pearson: 1.6422e-15 - mean_squared - ETA: 4s - loss: 425.2221 - pearson: 1.6432e-15 - mean_squared_error:  - ETA: 2s - loss: 448.9940 - pearson: 0.0114 - mean_squared_e - ETA: 0s - loss: 475.5805 - pearson: 0.0089 - mean_squared_error: 475.5805 - mean_absolute_err
Epoch 00016: val_loss improved from 203.50801 to 203.08665, saving model to models/mse_wide/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of

...

Epoch 31/100
7488/7500 [============================>.] - ETA: 0s - loss: 469.1399 - pearson: 0.0171 - mean_squared_error: 469.1399 - mean_absolute_error: 8.1943     ETA: 9s - loss: 427.4934 -  - ETA: 5s - loss: 444.3168 - pearson: 2.8303e-15 - mean_squared_error: 444.3168 - mean_absolute_error - ETA: 5s - loss: 440.3610 - pearson: 2.7114e-15 - mean_squared_error - ETA: 3s - loss: 444.6753 - pearson: 2.8324e-15 
Epoch 00031: val_loss improved from 200.11491 to 200.10909, saving model to models/mse_wide/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============

Epoch 46/100
7488/7500 [============================>.] - ETA: 0s - loss: 461.1396 - pearson: 0.0427 - mean_squared_error: 461.1396 - mean_absolute_error: 8.7270
Epoch 00046: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.4672 - pearson: 0.0427 - mean_squared_error: 461.4672 - mean_absolute_error: 8.7291 - val_loss: 201.2805 - val_pearson: 0.0016 - val_mean_squared_error: 201.2805 - val_mean_absolute_error: 6.3207
Epoch 47/100
7488/7500 [============================>.] - ETA: 0s - loss: 462.0538 - pearson: 0.0684 - mean_squared_error: 462.0538 - mean_absolute_error: 8.7408
Epoch 00047: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.4051 - pearson: 0.0683 - mean_squared_error: 461.4051 - mean_absolute_error: 8.7353 - val_loss: 201.2895 - val_pearson: 1.8735e-15 - val_mean_squared_error: 201.2895 - val_mean_absolute_error: 6.3238
Epoch 48/100
7488/7500 [============================>.] - ETA: 0

Epoch 63/100
7488/7500 [============================>.] - ETA: 0s - loss: 461.5645 - pearson: 0.0684 - mean_squared_error: 461.5646 - mean_absolute_error: 8.7559
Epoch 00063: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.2088 - pearson: 0.0683 - mean_squared_error: 461.2088 - mean_absolute_error: 8.7547 - val_loss: 201.3687 - val_pearson: 2.6110e-15 - val_mean_squared_error: 201.3687 - val_mean_absolute_error: 6.3511
Epoch 64/100
7488/7500 [============================>.] - ETA: 0s - loss: 459.8369 - pearson: 0.0598 - mean_squared_error: 459.8369 - mean_absolute_error: 8.7450
Epoch 00064: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.2031 - pearson: 0.0597 - mean_squared_error: 461.2031 - mean_absolute_error: 8.7553 - val_loss: 201.3711 - val_pearson: 2.2766e-15 - val_mean_squared_error: 201.3711 - val_mean_absolute_error: 6.3519
Epoch 65/100
7488/7500 [============================>.] - ET

1000/1000 [==============================] - 1s 692us/steps: 460.8365 - pearson: 0.0684 - mean_squared_error: 460.8365 - mean_absolute_e
[('loss', 260.51744769287109), ('pearson', 3.3360711674316168e-15), ('mean_squared_error', 260.51745150756835), ('mean_absolute_error', 6.9861587181091309)]
Test spearman: nan

Epoch 00080: val_loss did not improve
7500/7500 [==============================] - 22s 3ms/step - loss: 461.1167 - pearson: 0.0683 - mean_squared_error: 461.1167 - mean_absolute_error: 8.7640 - val_loss: 201.4088 - val_pearson: 2.4649e-15 - val_mean_squared_error: 201.4088 - val_mean_absolute_error: 6.3646
Epoch 81/100
7488/7500 [============================>.] - ETA: 0s - loss: 460.0470 - pearson: 0.0342 - mean_squared_error: 460.0470 - mean_absolute_error: 8.7599
Epoch 00081: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.1114 - pearson: 0.0341 - mean_squared_error: 461.1114 - mean_absolute_error: 8.7645 - val_loss: 201.4112 - v

Epoch 97/100
7488/7500 [============================>.] - ETA: 0s - loss: 460.8523 - pearson: 0.0598 - mean_squared_error: 460.8523 - mean_absolute_error: 8.7678
Epoch 00097: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.0253 - pearson: 0.0597 - mean_squared_error: 461.0253 - mean_absolute_error: 8.7732 - val_loss: 201.4495 - val_pearson: 2.4267e-15 - val_mean_squared_error: 201.4495 - val_mean_absolute_error: 6.3781
Epoch 98/100
7488/7500 [============================>.] - ETA: 0s - loss: 461.3426 - pearson: 0.0256 - mean_squared_error: 461.3426 - mean_absolute_error: 8.7740
Epoch 00098: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 461.0197 - pearson: 0.0256 - mean_squared_error: 461.0197 - mean_absolute_error: 8.7738 - val_loss: 201.4521 - val_pearson: 2.9393e-15 - val_mean_squared_error: 201.4521 - val_mean_absolute_error: 6.3790
Epoch 99/100
7488/7500 [============================>.] - ET